<a href="https://colab.research.google.com/github/nauman-akram/textfilesACL/blob/master/IR-System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


sample_data


In [0]:
#!unzip 'ACL txt.zip'

In [0]:
%cd /gdrive/My Drive/Classroom/Information Retrieval & Text Mining Fall 2019/

/gdrive/My Drive/Classroom/Information Retrieval & Text Mining Fall 2019


In [0]:
%cd ACL_txt

/gdrive/My Drive/Classroom/Information Retrieval & Text Mining Fall 2019/ACL_txt


In [0]:
import os
directory = os.getcwd()
directory

'/gdrive/My Drive/Classroom/Information Retrieval & Text Mining Fall 2019/ACL_txt'

In [0]:
def wordList(doc):
    fileHandler= open(doc,encoding='Latin-1')
    content = fileHandler.read()
    fileHandler.close()
    return content.split()
            

In [0]:
from gensim import utils
import gensim.parsing.preprocessing as gsp

filters = [
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.remove_stopwords,
          ]

def removePunc(wordlist):
    req_list = []
    for word in wordlist:
        word = word.lower()
        for fltr in filters:
            word = fltr(word)
        if len(word) != 0:
            req_list.append(word)
    return req_list



In [0]:
def termFrequencyInDoc(words_List):
    doc_dict = dict()
    for word in words_List:
        if word in doc_dict:
            doc_dict[word] += 1
        else:
            doc_dict[word] = 1
    return doc_dict


In [0]:
def wordDocFreq(dictList):
    docFreq = dict()
    for docDict in dictList:
        for word in docDict.keys():
            if word in docFreq:
                docFreq[word] += 1
            else:
                docFreq[word] = 1
    return docFreq





In [0]:
import numpy as np
def inverseDocFreq(dictList_wdf,M):    
    
    idf_corpus = {}
    for word in dictList_wdf.keys():
        idf_corpus[word] = np.log2((M+1) / dictList_wdf[word]) #log_2 ((M+1)/k)
    
    return idf_corpus

In [0]:
def tfidf(docsList):
    M = len(docsList)
    tf_dictList = []
    x=0
    for doc_path in docsList:
        x += 1
        words_List = wordList(doc_path) #calling first function to tokenize words
        clean_words_List = removePunc(words_List) #cleaning words form punc,\n,stopwords, get in lower value
        doc_dict = termFrequencyInDoc(clean_words_List) #calculate termfreqs in each document
        tf_dictList.append(doc_dict) #make list of dictionaries; every dictionary contain term freqs of words in document ##TF
        if x % 500 == 0:
            print("In loop",x,"files have been processed!")   

    wdf = wordDocFreq(tf_dictList)
    print("word-document frequency is calculated for whole vocab!")
    idfs = inverseDocFreq(wdf,M)

    print("Inv w-d frequency is calculated for whole vocab!")
    #print(idfs)
    #calulate tf*idfs of each word in each document make ditionary for every document!
    tf_idfs = [] #list of dictionaries containing tf-ids
    for docDict in tf_dictList:
        dict_doc_tfidf = dict()
        for word in docDict.keys():
            dict_doc_tfidf[word] = docDict[word] * idfs[word]
        tf_idfs.append(dict_doc_tfidf)
    
    vocab = idfs.keys()

    return tf_dictList, tf_idfs, vocab


In [0]:
def get_dictionaries(docsList,dictList,tf_idfs):
    names = []
    for doc_path in docsList:
        names.append(os.path.basename(doc_path))
    
    tf_idf_docs_dict = dict(zip(names,tf_idfs))
    tf_docs_dict = dict(zip(names,dictList))
    return  tf_docs_dict, tf_idf_docs_dict

In [0]:
def VectorSpaceModel (query,tf_idf_docs_dict, vocab):   
    query_vocab = [] 
    for word in query.split():
        if word not in query_vocab:
            query_vocab.append(word.lower())
    
    query_wc = {} # a dictionary to store count of a word in the query (i.e x_i according to lecture slides terminology)
    #print(query_vocab)
    query = query.lower()
    for word in query_vocab:
        query_wc[word] = query.split().count(word)
    #print(query_wc)
    relevance_scores = {} 

    for doc_name in tf_idf_docs_dict.keys():
        score = 0 
        for word in query_vocab:
            if word in tf_idf_docs_dict[doc_name].keys():
                score += query_wc[word] * tf_idf_docs_dict[doc_name][word]
            else: 
                continue
        relevance_scores[doc_name] = score
    
   
    top_5 = sorted(relevance_scores, key=relevance_scores.get, reverse=True)[:5]
    print("Document Relevancy Scores\n")
    print(query,"belongs to: ")
    for i in top_5:
        print("file",i," , with score: ",np.round(relevance_scores[i],2) )

    return relevance_scores

In [0]:
list_files = []
for files in os.walk(directory):
  for file in files[-1]:
    list_files.append(os.path.join(directory,file))

            

In [0]:
tf_dictList, tf_idfs, vocab = tfidf(list_files)

In loop 500 files have been processed!
In loop 1000 files have been processed!
In loop 1500 files have been processed!
In loop 2000 files have been processed!
In loop 2500 files have been processed!
In loop 3000 files have been processed!
In loop 3500 files have been processed!
In loop 4000 files have been processed!
In loop 4500 files have been processed!
In loop 5000 files have been processed!
In loop 5500 files have been processed!
In loop 6000 files have been processed!
In loop 6500 files have been processed!
In loop 7000 files have been processed!
In loop 7500 files have been processed!
In loop 8000 files have been processed!
In loop 8500 files have been processed!
In loop 9000 files have been processed!
In loop 9500 files have been processed!
In loop 10000 files have been processed!
In loop 10500 files have been processed!
In loop 11000 files have been processed!
In loop 11500 files have been processed!
In loop 12000 files have been processed!
In loop 12500 files have been proces

In [0]:
tf_docs_dict, tf_idf_docs_dict = get_dictionaries(list_files,tf_dictList,tf_idfs)

In [0]:
query1 = "Text Mining"
query2 = "LDA"
query3 = "topic modelling"
query4 = "Natural language Processing"
query5 = "generative Models"


In [0]:
rel_score_all = VectorSpaceModel(query1,tf_idf_docs_dict,vocab)

Document Relevancy Scores

text mining belongs to: 
file D09-1162.pdf.txt  , with score:  163.19
file P06-1062.pdf.txt  , with score:  162.88
file P12-1062.pdf.txt  , with score:  123.7
file W09-2609.pdf.txt  , with score:  120.4
file P09-1098.pdf.txt  , with score:  111.21


In [0]:
len(vocab)

1882865

In [0]:
rel_score_all = VectorSpaceModel(query2,tf_idf_docs_dict,vocab)

Document Relevancy Scores

lda belongs to: 
file J14-2003.pdf.txt  , with score:  381.57
file D09-1026.pdf.txt  , with score:  352.22
file D11-1050.pdf.txt  , with score:  342.44
file N10-1070.pdf.txt  , with score:  313.09
file P10-1117.pdf.txt  , with score:  298.41


In [0]:
rel_score_all = VectorSpaceModel(query3,tf_idf_docs_dict,vocab)

Document Relevancy Scores

topic modelling belongs to: 
file J14-2003.pdf.txt  , with score:  380.15
file P12-1079.pdf.txt  , with score:  337.71
file Q15-1004.pdf.txt  , with score:  302.64
file N15-1074.pdf.txt  , with score:  300.8
file W10-4104.pdf.txt  , with score:  280.5


In [0]:
rel_score_all  = VectorSpaceModel(query4,tf_idf_docs_dict,vocab)

Document Relevancy Scores

natural language processing belongs to: 
file W11-0100.pdf.txt  , with score:  154.24
file J14-1005.pdf.txt  , with score:  89.61
file J87-1020.pdf.txt  , with score:  86.23
file W14-55.x.pdf.txt  , with score:  61.91
file J86-2001.pdf.txt  , with score:  55.93


In [0]:
rel_score_all = VectorSpaceModel(query5,tf_idf_docs_dict,vocab)

Document Relevancy Scores

generative models belongs to: 
file W06-1668.pdf.txt  , with score:  187.97
file W11-0100.pdf.txt  , with score:  176.97
file J03-4003.pdf.txt  , with score:  150.44
file D09-1111.pdf.txt  , with score:  133.06
file D09-1058.pdf.txt  , with score:  132.46
